<a href="https://colab.research.google.com/github/JakeOh/202511_BD53/blob/main/lab_ml/ml03_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   훈련 셋과 테스트 셋을 나누는 방법
    *   순차 추출(sequential sampling)
    *   임의 추출(random sampling)
    *   층화 추출(stratified sampling)
*   분류 모델의 평가 지표(metrics)
    *   정확도(accuracy)
    *   정밀도(precision)
    *   재현율(recall)
    *   F1 score
*   특성 스케일링(feature scaling)
    *   표준화(standardization)
    *   정규화(normalization)

# 모듈 임포트

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier  # 머신러닝 모델
from sklearn.model_selection import train_test_split  # 훈련/테스트 셋 분리 함수
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # 평가 지표 계산 함수
from sklearn.metrics import classification_report, confusion_matrix  # 평가 지표 내용 출력
from sklearn.preprocessing import StandardScaler, MinMaxScaler  # 특성 스케일링 변환기

# 데이터 준비

In [2]:
file_path = 'https://bit.ly/fish_csv_data'

In [3]:
fish = pd.read_csv(file_path)

In [4]:
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [5]:
fish.tail()

,Species,Weight,Length,Diagonal,Height,Width
154,Smelt,12.2,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,14.3,15.2,2.8728,2.0672
158,Smelt,19.9,15.0,16.2,2.9322,1.8792


In [6]:
pd.set_option('display.max_rows', 10)

In [7]:
df = fish[fish.Species.isin(['Bream', 'Smelt'])]  # Bream(도미)와 Smelt(빙어) 선택

In [8]:
df

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...
154,Smelt,12.2,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,14.3,15.2,2.8728,2.0672


In [9]:
x = df[['Weight', 'Length']].values  # 특성 배열(features)

In [10]:
x[:5]

array([[242. ,  25.4],
       [290. ,  26.3],
       [340. ,  26.5],
       [363. ,  29. ],
       [430. ,  29. ]])

In [11]:
x[-5:]

array([[12.2, 12.2],
       [13.4, 12.4],
       [12.2, 13. ],
       [19.7, 14.3],
       [19.9, 15. ]])

In [12]:
y = df.Species.values  # 타겟 배열(target)

In [13]:
y

array(['Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt',
       'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt'],
      dtype=object)

# 훈련 셋/테스트 셋 분리

*   머신 러닝(Machine Learning): 컴퓨터가 데이터를 스스로 학습해서 예측.
*   훈련(train)/학습(learning): 머신 러닝 알고리즘(모델)에게 특성 데이터(와 레이블)을 제공.
*   평가: 학습된 데이터와 학습되지 않은 데이터를 얼마나 잘 예측하는 지 점수화.
*   훈련 셋: 머신 러닝 모델에게 제공할 특성 데이터(와 레이블)
*   테스트 셋: 학습되지 않은 데이터를 얼마나 잘 예측하는 지 평가하기 위해서 남겨두는 데이터(와 레이블)